In [2]:
!pip install -q sklearn
!pip install pyarrow

     |████████████████████████████████| 17.7 MB 7.1 MB/s eta 0:00:01███████████▌      | 14.1 MB 7.1 MB/s eta 0:00:01ta 0:00:01


In [40]:
import numpy as np
import pandas as pd

import tensorflow as tf

from tensorflow import feature_column
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np

PREPATH = "/project/Alexander/Datasets_RAW"
df2017 = [
    "/Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv",
    "/Friday-WorkingHours-Afternoon-PortScan.pcap_ISCX.csv",
    "/Friday-WorkingHours-Morning.pcap_ISCX.csv",
    "/Monday-WorkingHours.pcap_ISCX.csv",
    "/Thursday-WorkingHours-Afternoon-Infilteration.pcap_ISCX.csv",
    "/Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX.csv",
    "/Tuesday-WorkingHours.pcap_ISCX.csv",
    "/Wednesday-workingHours.pcap_ISCX.csv"
]
dfs_names = [df2017]

In [41]:
dfs = {}
i = 0
for dataset in dfs_names:
    for path in dataset:
        dfs[i] = pd.read_csv(f'{PREPATH}{path}', low_memory=False, header=0, names= ["Flow ID", "Source IP", "Source Port", "Destination IP","destination port", "protocol", "timestamp", "flow duration", "total fwd packets", "total backward packets", "total length of fwd packets", "total length of bwd packets", "fwd packet length max", "fwd packet length min", "fwd packet length mean", "fwd packet length std", "bwd packet length max", "bwd packet length min", "bwd packet length mean", "bwd packet length std", "flow bytes/s", "flow packets/s", "flow iat mean", "flow iat std", "flow iat max", "flow iat min", "fwd iat total", "fwd iat mean", "fwd iat std", "fwd iat max", "fwd iat min", "bwd iat total", "bwd iat mean", "bwd iat std", "bwd iat max", "bwd iat min", "fwd psh flags", "bwd psh flags", "fwd urg flags", "bwd urg flags", "fwd header length", "bwd header length", "fwd packets/s", "bwd packets/s", "min packet length", "max packet length", "packet length mean", "packet length std", "packet length variance", "fin flag count", "syn flag count", "rst flag count", "psh flag count", "ack flag count", "urg flag count", "cwe flag count", "ece flag count", "down/up ratio", "average packet size", "avg fwd segment size", "avg bwd segment size", "fwd avg bytes/bulk", "fwd avg packets/bulk", "fwd avg bulk rate", "bwd avg bytes/bulk", "bwd avg packets/bulk", "bwd avg bulk rate", "subflow fwd packets", "subflow fwd bytes", "subflow bwd packets", "subflow bwd bytes", "init_win_bytes_forward", "init_win_bytes_backward", "act_data_pkt_fwd", "min_seg_size_forward", "active mean", "active std", "active max", "active min", "idle mean", "idle std", "idle max", "idle min", "label"])
        dfs[i] = dfs[i][["Source IP", "Source Port", "Destination IP","destination port", "protocol", "timestamp", "flow duration", "fin flag count", "syn flag count", "rst flag count", "psh flag count", "ack flag count", "urg flag count","total fwd packets", "total backward packets","total length of fwd packets", "total length of bwd packets","label"]]
        i+=1

In [42]:
import re

regex_patDOS = re.compile(r'.*dos.*', flags=re.IGNORECASE)
regex_patBruteForce = re.compile(r'.*brute.*', flags=re.IGNORECASE)
for i, path in dfs.items():
    dfs[i] = dfs[i].replace(['slowloris','hulk','Bot','slowbody2','slowread','goldeneye','slowheaders','rudy','TFTP','Syn','UDP-lag','Portmap','UDP','UDPLag'], "DoS")
    dfs[i] = dfs[i].replace(regex_patBruteForce, "BruteForce")
    dfs[i] = dfs[i].replace(regex_patDOS, "DoS")
    dfs[i] = dfs[i].replace(['SQL Injection','NetBIOS','LDAP','MSSQL','Heartbleed','FTP-Patator','SSH-Patator'], "BruteForce")
    dfs[i] = dfs[i].replace(['.*XSS.*','.*Sql.*'], "DoS",regex=True)
    dfs[i] = dfs[i].replace(['BENIGN','Benign'], "normal")
    dfs[i] = dfs[i][dfs[i].label != 'Infiltration']
    dfs[i] = dfs[i][dfs[i].label != 'Infilteration']

for key, path in dfs.items():
    dfs[key]['flags'] = ""
    dfs[key]['flags'] = np.where(dfs[key]['fin flag count'] > 0 , dfs[key].flags + "FIN ", dfs[key].flags)
    dfs[key]['flags'] = np.where(dfs[key]['syn flag count'] > 0 , dfs[key].flags + "SYN ", dfs[key].flags)
    dfs[key]['flags'] = np.where(dfs[key]['rst flag count'] > 0 , dfs[key].flags + "RST ", dfs[key].flags)
    dfs[key]['flags'] = np.where(dfs[key]['psh flag count'] > 0 , dfs[key].flags + "PSH ", dfs[key].flags)
    dfs[key]['flags'] = np.where(dfs[key]['ack flag count'] > 0 , dfs[key].flags + "ACK ", dfs[key].flags)
    dfs[key]['flags'] = np.where(dfs[key]['urg flag count'] > 0 , dfs[key].flags + "URG ", dfs[key].flags)
    dfs[key] = dfs[key].drop(columns = ['fin flag count', 'syn flag count', 'rst flag count', 'psh flag count', 'ack flag count','urg flag count'])

In [43]:
def reduce_mem_usage(props):
    start_mem_usg = props.memory_usage().sum() / 1024**2 
    NAlist = [] # Keeps track of columns that have missing values filled in. 
    for col in props.columns:
        if props[col].dtype != object and props[col].dtype != 'datetime64[ns]':  # Exclude strings
            
            # make variables for Int, max and min
            IsInt = False
            mx = props[col].max()
            mn = props[col].min()
            
            # Integer does not support NA, therefore, NA needs to be filled
            if not np.isfinite(props[col]).all(): 
                NAlist.append(col)
                props[col].fillna(mn-1,inplace=True)  
                   
            # test if column can be converted to an integer
            asint = props[col].fillna(0).astype(np.int64)
            result = (props[col] - asint)
            result = result.sum()
            if result > -0.01 and result < 0.01:
                IsInt = True

            
            # Make Integer/unsigned Integer datatypes
            if IsInt:
                if mn >= 0:
                    if mx < 255:
                        props[col] = props[col].astype(np.uint8)
                    elif mx < 65535:
                        props[col] = props[col].astype(np.uint16)
                    elif mx < 4294967295:
                        props[col] = props[col].astype(np.uint32)
                    else:
                        props[col] = props[col].astype(np.uint64)
                else:
                    if mn > np.iinfo(np.int8).min and mx < np.iinfo(np.int8).max:
                        props[col] = props[col].astype(np.int8)
                    elif mn > np.iinfo(np.int16).min and mx < np.iinfo(np.int16).max:
                        props[col] = props[col].astype(np.int16)
                    elif mn > np.iinfo(np.int32).min and mx < np.iinfo(np.int32).max:
                        props[col] = props[col].astype(np.int32)
                    elif mn > np.iinfo(np.int64).min and mx < np.iinfo(np.int64).max:
                        props[col] = props[col].astype(np.int64)    
            
            # Make float datatypes 32 bit
            else:
                props[col] = props[col].astype(np.float32)
    mem_usg = props.memory_usage().sum() / 1024**2 
    return props, NAlist

In [44]:
for i, path in dfs.items():
    dfs[i]['packets'] =  dfs[i]["total fwd packets"] +  dfs[i]["total backward packets"]
    dfs[i]['bytes'] =  dfs[i]['total length of fwd packets'] +  dfs[i]['total length of bwd packets']
    dfs[i], NAlist = reduce_mem_usage(dfs[i])

In [45]:
from sklearn import preprocessing
attack_types = []
for i, path in dfs.items():
    attack_types = list(dfs[i]["label"].unique()) + attack_types
u, indices = np.unique(attack_types, return_inverse=True)
print(u)

['BruteForce' 'DoS' 'PortScan' 'normal']


In [46]:
import ipaddress
for i, path in dfs.items():
    dfs[i]['timestamp'] = dfs[i]['timestamp'].astype(dtype='datetime64[ns]')
    dfs[i]['timestamp'] = dfs[i]['timestamp'].astype(int)
    dfs[i]['Source IP'] = int(ipaddress.ip_address(dfs[i]['Source IP'][1]))
    dfs[i]['Destination IP'] = int(ipaddress.ip_address(dfs[i]['Destination IP'][1]))
    dfs[i] = dfs[i].drop(columns = ['total fwd packets', 'total backward packets', 'total length of fwd packets', 'total length of bwd packets'])
    dfs[i] = dfs[i].replace(['BruteForce','DoS','PortScan'], "attack")

In [49]:
print(dfs[6])

         Source IP  Source Port  Destination IP  destination port  protocol  \
0       3232238085        49182      3232238083                88         6   
1       3232238085        49183      3232238083                88         6   
2       3232238085        49184      3232238083                88         6   
3       3232238085        49185      3232238083                88         6   
4       3232238085        49186      3232238083                88         6   
...            ...          ...             ...               ...       ...   
445904  3232238085        51694      3232238083                53        17   
445905  3232238085          443      3232238083             59317         6   
445906  3232238085        62256      3232238083                53        17   
445907  3232238085          443      3232238083             54726         6   
445908  3232238085        51801      3232238083                53        17   

                  timestamp  flow duration  fin fla

In [48]:
teller = 0
i = 0
for dataset in dfs_names:
    for path in dataset:
        dfs[i] = dfs[i].reset_index(drop=True)
        dfs[i].to_feather('./Datasets_RAW/Feather/'+path.split('/')[2].split('.')[0])
        i+=1

IndexError: list index out of range

In [38]:
dfs = {}
i = 0 
for dataset in dfs_names:
    for path in dataset:
        index = path.split('.')[0]
        #dfs[i] = pd.read_feather('./Datasets_RAW/Feather'+ index)
        i += 1

FileNotFoundError: [Errno 2] Failed to open local file './Datasets_RAW/Feather/Friday-WorkingHours-Afternoon-DDos'. Detail: [errno 2] No such file or directory

In [50]:
from sklearn import preprocessing 

dataframe = dfs[6]
label_encoder = preprocessing.LabelEncoder() 
dataframe['label'] = label_encoder.fit_transform(dataframe['label']) 
dataframe['target'] = dataframe['label']
dataframe = dataframe.drop(columns=['label'])

In [51]:
print(dataframe['target'].unique())

[1 0]


In [52]:
dataframe.columns = dataframe.columns.str.replace(' ', '_')
print(dataframe.columns)

Index(['Source_IP', 'Source_Port', 'Destination_IP', 'destination_port',
       'protocol', 'timestamp', 'flow_duration', 'fin_flag_count',
       'syn_flag_count', 'rst_flag_count', 'psh_flag_count', 'ack_flag_count',
       'urg_flag_count', 'packets', 'bytes', 'target'],
      dtype='object')


In [53]:
train, test = train_test_split(dataframe, test_size=0.33)
print(len(train), 'train examples')
print(len(test), 'test examples')

298759 train examples
147150 test examples


In [54]:
# A utility method to create a tf.data dataset from a Pandas Dataframe
def df_to_dataset(dataframe, shuffle=True, batch_size=500):
    dataframe = dataframe.copy()
    labels = dataframe.pop('target')
    print(labels)
    ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
    if shuffle:
        ds = ds.shuffle(buffer_size=len(dataframe))
    ds = ds.batch(batch_size)
    return ds

In [55]:
feature_columns = []

# numeric cols
for header in dataframe.columns:
    if header!='target':
        feature_columns.append(feature_column.numeric_column(header))

In [56]:
batch_size = 500
train_ds = df_to_dataset(train, batch_size=batch_size)
test_ds = df_to_dataset(test, shuffle=False, batch_size=batch_size)

209624    1
429012    1
291688    1
101792    1
411550    1
         ..
55149     1
12474     1
91193     1
213296    1
206394    1
Name: target, Length: 298759, dtype: int64
138561    1
94415     1
12331     1
344348    1
179184    1
         ..
175805    1
356089    1
42247     1
313136    1
122764    1
Name: target, Length: 147150, dtype: int64


In [60]:
from tensorflow import expand_dims

model = tf.keras.Sequential([
    tf.keras.layers.DenseFeatures(feature_columns),
    tf.keras.layers.Reshape((-1, 15), input_shape=(15,)),
    layers.LSTM(6, return_sequences = True),
    layers.Dense(1, activation= 'softmax')
])

opt = tf.keras.optimizers.RMSprop(learning_rate=0.01, rho=0.05, momentum=0.0, epsilon=1e-08, centered=False, name='RMSprop')
model.compile(optimizer=opt, loss = 'categorical_crossentropy', metrics=['accuracy'])

model.fit(train_ds, validation_data=test_ds, epochs=5, batch_size=500)

Epoch 1/5
598/598 [==============================] - 19s 32ms/step - loss: 1.1551e-07 - accuracy: 0.9689 - val_loss: 1.1552e-07 - val_accuracy: 0.9690
Epoch 2/5
598/598 [==============================] - 19s 32ms/step - loss: 1.1551e-07 - accuracy: 0.9689 - val_loss: 1.1552e-07 - val_accuracy: 0.9690
Epoch 3/5
598/598 [==============================] - 26s 43ms/step - loss: 1.1551e-07 - accuracy: 0.9689 - val_loss: 1.1552e-07 - val_accuracy: 0.9690
Epoch 4/5
598/598 [==============================] - 16s 26ms/step - loss: 1.1551e-07 - accuracy: 0.9689 - val_loss: 1.1552e-07 - val_accuracy: 0.9690
Epoch 5/5
598/598 [==============================] - 15s 25ms/step - loss: 1.1551e-07 - accuracy: 0.9689 - val_loss: 1.1552e-07 - val_accuracy: 0.9690


In [61]:
y_true = test['target']
y_true = y_true.to_list()
X = test.drop(columns=['target'])
y = model.predict(X)
y_round = tf.make_ndarray(tf.make_tensor_proto(tf.round(y))).reshape([len(y)])
y_round = list(y_round)
print("Gebalanceerde accuracy van model met een ongekende dataset:")
print(str(int(balanced_accuracy_score(y_true, y_round)*10000)/100) + "%")

ValueError: in user code:

    /opt/conda/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:1147 predict_function  *
        outputs = self.distribute_strategy.run(
    /opt/conda/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:951 run  **
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /opt/conda/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:2290 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /opt/conda/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:2649 _call_for_each_replica
        return fn(*args, **kwargs)
    /opt/conda/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:1122 predict_step  **
        return self(x, training=False)
    /opt/conda/lib/python3.7/site-packages/tensorflow/python/keras/engine/base_layer.py:927 __call__
        outputs = call_fn(cast_inputs, *args, **kwa

In [20]:
model.save_weights('./checkpoints/checkpoint')

In [21]:
print(dfs[6])

         Source IP  Source Port  Destination IP  destination port  protocol  \
0       3232238085        49182      3232238083                88         6   
1       3232238085        49183      3232238083                88         6   
2       3232238085        49184      3232238083                88         6   
3       3232238085        49185      3232238083                88         6   
4       3232238085        49186      3232238083                88         6   
...            ...          ...             ...               ...       ...   
445904  3232238085        51694      3232238083                53        17   
445905  3232238085          443      3232238083             59317         6   
445906  3232238085        62256      3232238083                53        17   
445907  3232238085          443      3232238083             54726         6   
445908  3232238085        51801      3232238083                53        17   

                  timestamp  flow duration  fin fla